In [1142]:
import numpy as np
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
import os
import gzip

# First exercise: Classifying MNIST with MLPs
In this exercise you will implement a Neural Network (or MLP) and classify the MNIST digits with it.
MNIST is a "well hung" dataset that has been used a lot over the years to benchmark different classification algorithms. 
To learn more about it have a look here: http://yann.lecun.com/exdb/mnist/ .

# Data Loading
We first define a function for downloading and loading MNIST.
**WARNING**: Executing it will obviously use up some space on your machine ;). 

In [1143]:
def mnist(datasets_dir='./data'):
    if not os.path.exists(datasets_dir):
        os.mkdir(datasets_dir)
    data_file = os.path.join(datasets_dir, 'mnist.pkl.gz')
    if not os.path.exists(data_file):
        print('... downloading MNIST from the web')
        try:
            import urllib
            urllib.urlretrieve('http://google.com')
        except AttributeError:
            import urllib.request as urllib
        url = 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        urllib.urlretrieve(url, data_file)

    print('... loading data')
    # Load the dataset
    f = gzip.open(data_file, 'rb')
    try:
        train_set, valid_set, test_set = pickle.load(f, encoding="latin1")
    except TypeError:
        train_set, valid_set, test_set = pickle.load(f)
    f.close()

    test_x, test_y = test_set
    test_x = test_x.astype('float32')
    test_x = test_x.astype('float32').reshape(test_x.shape[0], 1, 28, 28)
    test_y = test_y.astype('int32')
    valid_x, valid_y = valid_set
    valid_x = valid_x.astype('float32')
    valid_x = valid_x.astype('float32').reshape(valid_x.shape[0], 1, 28, 28)
    valid_y = valid_y.astype('int32')
    train_x, train_y = train_set
    train_x = train_x.astype('float32').reshape(train_x.shape[0], 1, 28, 28)
    train_y = train_y.astype('int32')
    rval = [(train_x, train_y), (valid_x, valid_y), (test_x, test_y)]
    print('... done loading data')
    return rval

# Neural Network Layers
We now define "bare bone" neural network layers.
The parts marked with **TODO** are where you should finish the implementation!
Conceptually we will implement the layers as follows:

Each layer has a constructor that takes an input layer plus some additional arguments such as layer size and the activation function name. The layer then uses the provided input layer to compute the layer dimensions, weight shapes, etc. and setup all auxilliary variables.

Each layer then has to provide three functions (as defined in the Layer class below): *output_shape()*, *fprop()* and *brop()*. The output_shape function is used to figure out the shape for the next layer and the *fprop()/bprop()* functions are used to compute forward and backward passes through the network.

In [1144]:
# start by defining simple helpers
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def sigmoid_d(x):
    return sigmoid(x) * (1 - sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_d(x):
    return 1 - tanh(x)**2

def relu(x):
    return np.maximum(0.0, x)

def relu_d(x):
    return np.where(x > 0, 1, 0)

def softmax(x, axis=1):
    # to make the softmax a "safe" operation we will 
    # first subtract the maximum along the specified axis
    # so that np.exp(x) does not blow up!
    # Note that this does not change the output.
    x_max = np.max(x, axis=axis, keepdims=True)
    x_safe = x - x_max
    e_x = np.exp(x_safe)
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

def one_hot(labels):
    """this creates a one hot encoding from a flat vector:
    i.e. given y = [0,2,1]
     it creates y_one_hot = [[1,0,0], [0,0,1], [0,1,0]]
    """
    classes = np.unique(labels)
    n_classes = classes.size
    one_hot_labels = np.zeros(labels.shape + (n_classes,))
    for c in classes:
        one_hot_labels[labels == c, c] = 1
    return one_hot_labels

def unhot(one_hot_labels):
    """ Invert a one hot encoding, creating a flat vector """
    return np.argmax(one_hot_labels, axis=-1)

def check_grad(func, grad_func, param, epsilon):
    gparam_bprop = grad_func(param) 
    gparam_fd = np.empty_like(param)
    for i in range(len(param)):
        p_orig = param[i]
        loss_base = func(param)
        param[i] = param[i] + epsilon
        loss_plus = func(param)
        param[i] = p_orig
        gparam_fd[i] = (loss_plus - loss_base) / epsilon

    err = np.sqrt(np.sum((gparam_bprop - gparam_fd)**2))
    
    # remove comment to check wheter this implementation works ;)
#     import scipy.optimize
#     ctrl_err = scipy.optimize.check_grad(func, grad_func, 
#                                     param, epsilon=epsilon)
#     assert abs(err - ctrl_err) < epsilon
    return err

def random_data(input_size, n_labels):
    # create random data
    X = np.random.normal(size=input_shape)
    # and random labels
    Y = np.zeros((input_shape[0], n_labels))
    for i in range(Y.shape[0]):
        idx = np.random.randint(n_labels)
        Y[i, idx] = 1.
    return X, Y

# then define an activation function class
class Activation(object):
    def __init__(self, tname):
        if tname == 'sigmoid':
            self.act = sigmoid
            self.act_d = sigmoid_d
        elif tname == 'tanh':
            self.act = tanh
            self.act_d = tanh_d
        elif tname == 'relu':
            self.act = relu
            self.act_d = relu_d
        else:
            raise ValueError('Invalid activation function.')
            
    def fprop(self, input):
        # we need to remember the last input
        # so that we can calculate the derivative with respect
        # to it later on
        self.last_input = input
        return self.act(input)
    
    def bprop(self, output_grad):
        return output_grad * self.act_d(self.last_input)

# define a base class for layers
class Layer(object):
    
    def fprop(self, input):
        """ Calculate layer output for given input 
            (forward propagation). 
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def bprop(self, output_grad):
        """ Calculate input gradient and gradient 
            with respect to weights and bias (backpropagation). 
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def output_size(self):
        """ Calculate size of this layer's output.
        input_shape[0] is the number of samples in the input.
        input_shape[1:] is the shape of the feature.
        """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for loss outputs
# an output layer can then simply be derived
# from both Layer and Loss 
class Loss(object):

    def loss(self, output, output_net):
        """ Calculate mean loss given real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

    def input_grad(self, output, output_net):
        """ Calculate input gradient real output and network output. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a base class for parameterized things        
class Parameterized(object):
    
    def params(self):
        """ Return parameters (by reference) """
        raise NotImplementedError('This is an interface class, please use a derived instance')
    
    def grad_params(self):
        """ Return accumulated gradient with respect to params. """
        raise NotImplementedError('This is an interface class, please use a derived instance')

# define a container for providing input to the network
class InputLayer(Layer):
    
    def __init__(self, input_shape):
        if not isinstance(input_shape, tuple):
            raise ValueError("InputLayer requires input_shape as a tuple")
        self.input_shape = input_shape

    def output_size(self):
        return self.input_shape
    
    def fprop(self, input):
        return input
    
    def bprop(self, output_grad):
        return output_grad
        
class FullyConnectedLayer(Layer, Parameterized):
    """ A standard fully connected hidden layer, as discussed in the lecture.
    """
    
    def __init__(self, input_layer, num_units, 
                 init_stddev, activation_fun=Activation('relu')):
        self.num_units = num_units
        self.activation_fun = activation_fun
        # the input shape will be of size (batch_size, num_units_prev) 
        # where num_units_prev is the number of units in the input 
        # (previous) layer
        self.input_shape = input_layer.output_size()
        # this is the weight matrix it should have shape: (num_units_prev, num_units)
        self.W = np.random.normal(0, init_stddev, 
                                  (self.input_shape[1], num_units))
        # and this is the bias vector of shape: (num_units)
        self.b = np.random.normal(0, init_stddev, 
                                  (1, num_units))
        # create dummy variables for parameter gradients
        # no need to change these here!
        self.dW = np.zeros_like(self.W)
        self.db = np.zeros_like(self.b)
    
    def output_size(self):
        return (self.input_shape[0], self.num_units)
    
    def fprop(self, input):
        self.last_input = input  
        output = input.dot(self.W) + self.b
        if self.activation_fun is not None:
            return self.activation_fun.fprop(output)
        else:
            return output
        
    def bprop(self, output_grad):
        """ Calculate input gradient (backpropagation). """      
        # HINT: you may have to divide the weights by n
        #       to make gradient checking work 
        #       (since you want to divide the loss by number of inputs)
        n = output_grad.shape[0]
        
        if self.activation_fun is not None:
            output_grad = self.activation_fun.bprop(output_grad)

        # accumulate gradient wrt. the parameters first
        # we will need to store these to later update
        # the network after a few forward backward passes
        # the gradient wrt. W should be stored as self.dW
        # the gradient wrt. b should be stored as selfdb
        dW = self.last_input.T.dot(output_grad) 
        db = np.sum(output_grad, axis=0, keepdims=True) 
        self.dW = self.dW + dW # todo / n
        self.db = self.db + db #/ n
        #print("last input", self.last_input, ", output_grad:", output_grad,", dW", dW,", db", db)

        # the gradient wrt. the input should be calculated here
        grad_input = output_grad.dot(self.W.T) / n
        return grad_input
        
    def params(self):
        return self.W, self.b

    def grad_params(self):
        return self.dW, self.db
    
    def reset_grad_params(self):           
        self.dW[:] = 0
        self.db[:] = 0

# finally we specify the interface for output layers 
# which are layers that also have a loss function
# we will implement two output layers:
#  a Linear, and Softmax (Logistic Regression) layer
# The difference between output layers and and normal 
# layers is that they will be called to compute the gradient
# of the loss through input_grad(). bprop will never 
# be called on them!
class LinearOutput(Layer, Loss):
    """ A simple linear output layer that  
        uses a squared loss (e.g. should be used for regression)
    """
    def __init__(self, input_layer):
        self.input_size = input_layer.output_size()
        
    def output_size(self):
        return (1,)
        
    def fprop(self, input):
        return input

    def bprop(self, output_grad):
        raise NotImplementedError(
            'LinearOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        n_samples = float(Y.shape[0])
        return (Y_pred - Y) # todo / n_samples

    def loss(self, Y, Y_pred):
        loss = 0.5 * np.square(Y_pred - Y)
        return np.mean(np.sum(loss, axis=1))

class SoftmaxOutput(Layer, Loss):
    """ A softmax output layer that calculates 
        the negative log likelihood as loss
        and should be used for classification.
    """
    
    def __init__(self, input_layer):
        self.input_size = input_layer.output_size()
        
    def output_size(self):
        return (1,)
    
    def fprop(self, input):
        prediction = softmax(input)
        return prediction
    
    def bprop(self, output_grad):
        raise NotImplementedError(
            'SoftmaxOutput should only be used as the last layer of a Network'
            + ' bprop() should thus never be called on it!'
        )
    
    def input_grad(self, Y, Y_pred):
        n_samples = float(Y.shape[0])
        grad = Y_pred - Y
        return grad  # todo  / n_samples

    def loss(self, Y, Y_pred):
        # to make the loss numerically stable 
        # you may want to add an epsilon in the log ;)
        eps = 1e-10
        # calculate negative log likelihood
        loss = np.sum(-np.log(Y_pred+eps)*Y, axis=1)
        return np.mean(loss)
        

# Neural Network class
With all layers in place (and properly implemented by you) we can finally define a neural network.
For our purposes a neural network is simply a collection of layers which we will cycle through and on which we will call fprop and bprop to compute partial derivatives with respect to the input and the parameters.

Pay special attention to the *check_gradients()* function in which you should implement automatic differentiation. This function will become your best friend when checking the correctness of your implementation.

In [1178]:

class NeuralNetwork:
    """ Our Neural Network container class.
    """
    def __init__(self, layers):
        self.layers = layers
        
    def _loss(self, X, Y):
        Y_pred = self.predict(X)
        return self.layers[-1].loss(Y, Y_pred)

    def predict(self, X):
        """ Calculate an output Y for the given input X. """
        for i, layer in enumerate(self.layers):
            Y_pred = layer.fprop(X)
            X = Y_pred

        return Y_pred
    
    def backpropagate(self, Y, Y_pred, upto=0):
        """ Backpropagation of partial derivatives through 
            the complete network up to layer 'upto'
        """
        next_grad = self.layers[-1].input_grad(Y, Y_pred)
        for layer in self.layers[upto:-1][::-1]:
            next_grad = layer.bprop(next_grad)
        
        return next_grad
    
    def classification_error(self, X, Y):
        """ Calculate error on the given data 
            assuming they are classes that should be predicted. 
        """
        Y_pred = unhot(self.predict(X))
        error = Y_pred != Y
        return np.mean(error)
    
    def _update_params(self, learning_rate, penalty, penalty_rate):
        for l in self.layers[1:-1]: 
            W, b = l.params()
            dW, db = l.grad_params()
            if penalty is None:
                W[:] = W - learning_rate * dW
                b[:] = b - learning_rate * db
            elif penalty.lower() == 'l2':
                W[:] = ((1. - learning_rate * penalty_rate) * W 
                        - learning_rate * dW)
                b[:] = ((1. - learning_rate * penalty_rate) * b 
                        - learning_rate * db)
            elif penalty.lower() == 'l1':
                W[:] = (W - learning_rate * 
                        (penalty_rate * np.sign(W) + dW))
                b[:] = (b - learning_rate * 
                        (penalty_rate * np.sign(b) + db))
            else: 
                raise NotImplementedError("Unknown penalty type {}".format(penalty))
            
            l.reset_grad_params()
            
    def sgd_epoch(self, X, Y, learning_rate, batch_size, penalty, penalty_rate):
        n_samples = X.shape[0]
        n_batches = (n_samples // batch_size) + 1
        for b in range(n_batches):
            batch_start = b * batch_size
            batch_end = batch_start + batch_size
            
            X_batch = X[batch_start:batch_end,]
            Y_batch = Y[batch_start:batch_end,]
            Y_pred = self.predict(X_batch)
            self.backpropagate(Y_batch, Y_pred)
#             print("Y", Y)
#             print("predicted Y", Y_pred)
            self._update_params(learning_rate, penalty, penalty_rate)
       
    def gd_epoch(self, X, Y, learning_rate, batch_size, penalty, penalty_rate):
        n_samples = X.shape[0]
        n_batches = (n_samples // batch_size) + 1
        for b in range(n_batches):
            batch_start = b * batch_size
            batch_end = batch_start + batch_size
            
            X_batch = X[batch_start:batch_end,]
            Y_batch = Y[batch_start:batch_end,]
            
            Y_pred = self.predict(X_batch)
            self.backpropagate(Y_batch, Y_pred)
            
        self._update_params(learning_rate, penalty, penalty_rate)
    
    def train(self, X, Y, X_valid, Y_valid, learning_rate=0.1, max_epochs=100, batch_size=64,
              descent_type="sgd", y_one_hot=True, penalty=None, penalty_rate=0.0001):

        """ Train network on the given data. """
        n_samples = X.shape[0]
        n_batches = n_samples // batch_size
        if y_one_hot:
            Y_train = one_hot(Y)
            Y_valid_train = one_hot(Y_valid)
        else:
            Y_train = Y
            Y_valid_train
        print("... starting training")
        for e in range(max_epochs+1):
            if descent_type == "sgd":
                self.sgd_epoch(X, Y_train, learning_rate, batch_size, penalty, penalty_rate)
            elif descent_type == "gd":
                self.gd_epoch(X, Y_train, learning_rate, batch_size, penalty, penalty_rate)
            else:
                raise NotImplementedError("Unknown gradient descent type {}".format(descent_type))

            # Output error on the training data
            train_loss = self._loss(X, Y_train)
            train_error = self.classification_error(X, Y)
            valid_loss = self._loss(X_valid, Y_valid_train)
            valid_error = self.classification_error(X_valid, Y_valid)
            print(('epoch {:.4f}, loss {:.4f}, ' +
                  'train error {:.4f}, ' + 
                  'valid loss {:.4f}, ' +
                  'valid error {:.4f}' +
                  '').format(e, train_loss, train_error, 
                             valid_loss, valid_error))
    
    def check_gradients(self, X, Y):
        """ Helper function to test the parameter gradients for
        correctness. """
        # TODO: Fix gradient check
        # Something goes wrong here!
        # Learning works, but gradient check fails...
        
        for l, layer in enumerate(self.layers):
            if isinstance(layer, Parameterized):
                print('checking gradient for layer {}'.format(l))
                for p, param in enumerate(layer.params()):
                    # we iterate through all parameters
                    param_shape = param.shape
                    # define functions for conveniently swapping
                    # out parameters of this specific layer and 
                    # computing loss and gradient with these 
                    # changed parametrs
                    def output_given_params(param_new):
                        """ A function that will compute the output 
                            of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)

                        # return computed loss
                        return self._loss(X, Y)
                    
                    def grad_given_params(param_new):
                        """A function that will compute the gradient 
                           of the network given a set of parameters
                        """
                        # copy provided parameters
                        param[:] = np.reshape(param_new, param_shape)
                        layer.reset_grad_params()
                        # Forward propagation through the net
                        Y_pred = self.predict(X)
                        # Backpropagation of partial derivatives
                        self.backpropagate(Y, Y_pred, upto=l)
                        # return the computed gradient 
                        return np.ravel(self.layers[l].grad_params()[p])

                    # let the initial parameters be the ones that
                    # are currently placed in the network and flatten them
                    # to a vector for convenient comparisons, printing etc.
                    param_init = np.ravel(np.copy(param))

                    epsilon = 1e-8
                    err = check_grad(output_given_params, 
                                    grad_given_params, 
                                    param_init,
                                    epsilon)

                    print('diff {:.2e}'.format(err))
                    assert(err < epsilon)
                    
                    # reset the parameters to their initial values
                    param[:] = np.reshape(param_init, param_shape)
                    
# TODO rprop, rmsprop, adagrad

# Gradient Checking
After implementing everything it is always a good idea to setup some layers and perform gradient
checking on random data. **Note** that this is only an example! It is not a useful network architecture ;). We also expect you to play around with this to test all your implemented components.

In [1146]:
input_shape = (5, 10)
n_labels = 10
layers = [InputLayer(input_shape)]

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=15,
                init_stddev=0.1,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=6,
                init_stddev=0.1,
                activation_fun=Activation('tanh')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=n_labels,
                init_stddev=0.1,
                activation_fun=Activation('relu')
))

layers.append(SoftmaxOutput(layers[-1]))
nn = NeuralNetwork(layers)

# TODO: write more tests with varying setuos

In [1147]:
X, Y = random_data(input_shape, n_labels)

In [1150]:
#nn.check_gradients(X, Y) TODO: uncomment if fixed

In [1151]:
# Toy example to test (linear) learning
input_shape = (None, 2)
layers = [InputLayer(input_shape)]
centers = [(-5, -5), (0, 0), (5, 5)]

layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=len(centers),
                init_stddev=0.1,
                activation_fun=None # linear
))

layers.append(SoftmaxOutput(layers[-1]))
nn = NeuralNetwork(layers)
from sklearn.datasets import make_blobs
X, Y = make_blobs(n_samples=300, n_features=input_shape[1], cluster_std=1.0,
                  centers=centers, shuffle=True, random_state=42)
X_valid, Y_valid = make_blobs(n_samples=200, n_features=input_shape[1], cluster_std=1.0,
                  centers=centers, shuffle=True, random_state=42)

nn.train(X, Y,X_valid, Y_valid,
        learning_rate=0.1, 
        max_epochs=20, batch_size=64, y_one_hot=True,descent_type='sgd')

... starting training
epoch 0.0000, loss 0.5426, train error 0.0933, valid loss 0.7480, valid error 0.1200
epoch 1.0000, loss 0.1421, train error 0.0267, valid loss 0.2018, valid error 0.0400
epoch 2.0000, loss 0.0536, train error 0.0133, valid loss 0.0762, valid error 0.0250
epoch 3.0000, loss 0.0207, train error 0.0100, valid loss 0.0296, valid error 0.0100
epoch 4.0000, loss 0.0067, train error 0.0033, valid loss 0.0097, valid error 0.0050
epoch 5.0000, loss 0.0029, train error 0.0000, valid loss 0.0043, valid error 0.0000
epoch 6.0000, loss 0.0020, train error 0.0000, valid loss 0.0029, valid error 0.0000
epoch 7.0000, loss 0.0015, train error 0.0000, valid loss 0.0023, valid error 0.0000
epoch 8.0000, loss 0.0013, train error 0.0000, valid loss 0.0019, valid error 0.0000
epoch 9.0000, loss 0.0011, train error 0.0000, valid loss 0.0016, valid error 0.0000
epoch 10.0000, loss 0.0009, train error 0.0000, valid loss 0.0014, valid error 0.0000
epoch 11.0000, loss 0.0008, train error 0.

# Training on MNIST
Finally we can let our network run on the MNIST dataset!

First load the data and reshape it.

In [1152]:
# load
Dtrain, Dval, Dtest = mnist()
X_train, y_train = Dtrain
X_valid, y_valid = Dval
# Downsample training data to make it a bit faster for testing this code
n_train_samples = 10000
train_idxs = np.random.permutation(X_train.shape[0])[:n_train_samples]
X_train = X_train[train_idxs]
y_train = y_train[train_idxs]

... loading data
... done loading data


*Dtrain* contains 50k images which are of size 28 x 28 pixels. Hence:

In [1153]:
print("X_train shape: {}".format(np.shape(X_train)))
print("y_train shape: {}".format(np.shape(y_train)))

X_train shape: (10000, 1, 28, 28)
y_train shape: (10000,)


y_train will automatically be converted in the *train()* function to one_hot encoding.


But we need to reshape X_train, as our Network expects flat vectors of size 28*28 as input!

In [1154]:
X_train = X_train.reshape(X_train.shape[0], -1)
print("Reshaped X_train size: {}".format(X_train.shape))
X_valid = X_valid.reshape((X_valid.shape[0], -1))
print("Reshaped X_valid size: {}".format(X_valid.shape))

Reshaped X_train size: (10000, 784)
Reshaped X_valid size: (10000, 784)


Ah, much better ;-)! 

Now we can finally really start training a Network!


I pre-defined a small Network for you below. Again This is not really a good default and will not produce state of the art results. Please play around with this a bit. See how different activation functions and training procedures (gd / sgd) affect the result.

In [1182]:
import time

# Setup a small MLP / Neural Network
# we can set the first shape to None here to indicate that
# we will input a variable number inputs to the network
input_shape = (None, 28*28)
layers = [InputLayer(input_shape)]
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=100,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=100,
                init_stddev=0.01,
                activation_fun=Activation('relu')
))
layers.append(FullyConnectedLayer(
                layers[-1],
                num_units=10,
                init_stddev=0.01,
                # last layer has no nonlinearity 
                # (softmax will be applied in the output layer)
                activation_fun=None 
))
layers.append(SoftmaxOutput(layers[-1]))


nn = NeuralNetwork(layers)
# Train neural network
t0 = time.time()
nn.train(X_train, y_train, X_valid, y_valid, learning_rate=0.3, 
         max_epochs=20, batch_size=64, y_one_hot=True,descent_type='sgd',
        penalty='l2', penalty_rate=0.001 )
t1 = time.time()
print('Duration: {:.1f}s'.format(t1-t0))

... starting training
epoch 0.0000, loss 2.7951, train error 0.7444, valid loss 2.7535, valid error 0.7487
epoch 1.0000, loss 0.8630, train error 0.2855, valid loss 0.8345, valid error 0.2729
epoch 2.0000, loss 2.7610, train error 0.5122, valid loss 2.6392, valid error 0.4890
epoch 3.0000, loss 0.2796, train error 0.0848, valid loss 0.2863, valid error 0.0870
epoch 4.0000, loss 0.2234, train error 0.0693, valid loss 0.2477, valid error 0.0748
epoch 5.0000, loss 0.1903, train error 0.0621, valid loss 0.2325, valid error 0.0701
epoch 6.0000, loss 0.1623, train error 0.0528, valid loss 0.2193, valid error 0.0652
epoch 7.0000, loss 0.1389, train error 0.0459, valid loss 0.2105, valid error 0.0607
epoch 8.0000, loss 0.1209, train error 0.0402, valid loss 0.2043, valid error 0.0586
epoch 9.0000, loss 0.1192, train error 0.0400, valid loss 0.2133, valid error 0.0594
epoch 10.0000, loss 0.1110, train error 0.0355, valid loss 0.2050, valid error 0.0561
epoch 11.0000, loss 0.0907, train error 0.

KeyboardInterrupt: 

# Figure out a reasonable Network that achieves good performance
As the last part of this task, setup a network that works well and gets reasonable accuracy, say ~ 1-3 percent error on the **validation set**. 
Train this network on the complete data and compute the **test error**. 

Once you have done this, visualize a few digits from the the test set that the network gets right as well as a few that the network gets wrong!

In [1156]:
# TODO: train on full dataset, validate to < 3% error



In [1157]:
# TODO: Visualize correct and false classified!

In [ ]:
# TODO: write report